# IE 306.02 Assignment 1 

## Call Center Simulation

Imports of random, numpy and simpy:

In [ ]:
import random
import numpy
import simpy

Initializations of the parameters that characterize the service times of the front desk operator and expert. 

In [ ]:
RANDOM_SEED = 978

INTERARRIVAL_RATE = 14.3
MEAN_RENEGE =  60

MEAN_FRONT_DESK = 7.2
STD_FRONT_DEKS = 2.7

MEAN_EXPERT= 10.2

MEAN_BREAK = 60

BREAK_DURATION = 3

SERVICE_RANGE = [50, 90]
random.seed(RANDOM_SEED)

Initializations of the arrays that are going to be used for collecting data during the whole process.  
HIGH and LOW corresponds to the priorities which are used for "take break" operation.

In [ ]:
service_times_front_desk = [] 
service_times_expert = []

queue_w_times_front_desk = []
queue_w_times_expert = []

HIGH = 1
LOW = 2

CUSTOMER_NUMBER = 1000

### Customer Class Definition

In [ ]:
class Customer(object):

    total_customer = 0

    def __init__(self, name, env, front_desk_operator, expert_operator, action_break_generator):
        self.env = env
        self.name = name
        self.arrival_to_front_desk = None
        self.arrival_to_expert = None
        self.action = env.process(self.call())
        self.action_break_generator = action_break_generator
    
    def call(self):

        Customer.total_customer += 1 #incrementing the number of total customers when a call is arrived
        
        print('%s initiated a call at %g' % (self.name, self.env.now)) #prints the customer number and current time

        self.arrival_to_front_desk = self.env.now #customer now arrived to the front desk

        with front_desk_operator.request() as req:

            patience = random.expovariate(MEAN_RENEGE) #Renege time of a customer is exponentially distributed

            results = yield req | env.timeout(patience) #result is either equal to front desk operators request or renege time

            if req in results: #condition of customer being served by front desk operator
                print('%s is assigned to the front desk operator at %g' % (self.name, self.env.now))
                queue_w_times_front_desk.append(self.env.now - self.arrival_to_front_desk) #Append the waiting time of the current customer to the list of front desk queue
                yield self.env.process(self.get_front_desk_service()) #Process the customer's call to front desk operator
                print('%s is done with front desk operator at %g' % (self.name, self.env.now))
            else: #condition of reneged customer
                print('%s is reneged from the queue of the front desk operator at %g' % (self.name, self.env.now))
                env.exit()

        self.arrival_to_expert = self.env.now #customer now arrived to the expert

        with expert_operator.request(priority = HIGH) as req:
            
            patience = random.expovariate(MEAN_RENEGE) #Renege time of a customer is exponentially distributed

            results = yield req | env.timeout(patience) #result is either equal to expert operators request or renege time

            if req in results: #condition of customer being served by expert operator
                print('%s is assigned to the expert operator at %g' % (self.name, self.env.now))
                queue_w_times_expert.append(self.env.now - self.arrival_to_expert) #Append the waiting time of the current customer to the list of expert queue
                yield self.env.process(self.get_expert_service()) #Process the customer's call to expert operator
                print('%s is done with expert operator at %g' % (self.name, self.env.now))
            else: #condition of reneged customer
                print('%s is reneged from the queue of the expert operator at %g' % (self.name, self.env.now))
                env.exit()

        if Customer.total_customer == CUSTOMER_NUMBER and self.action_break_generator.is_alive: #if total customer number became 1000, then interrupt
          self.action_break_generator.interrupt()

    def get_front_desk_service(self): #the function of front desk operator service
        #duration = numpy.random.lognormal(MEAN_FRONT_DESK, STD_FRONT_DEKS)
        duration = random.expovariate(MEAN_EXPERT) #Duration of a call on the front desk is exponentially distributed
        yield self.env.timeout(duration) #Add the duration of a call on the front desk to the current time
        service_times_front_desk.append(duration) #Append the customer to the front desk operator

    def get_expert_service(self): # the function of expert operator service
        duration = random.expovariate(MEAN_EXPERT) #Duration of a call on the expert operator is exponentially distributed
        yield self.env.timeout(duration) #Add the duration of a call on the expert operator to the current time
        service_times_expert.append(duration) #Append the customer to the expert operator


#### Customer Generator Function

In [ ]:
def customer_generator(env, front_desk_operator, expert_operator, action_break_generator):
    for i in range(CUSTOMER_NUMBER):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE)) #Add the exponentially distributed interarrival times between the customers
        Customer('Cust %s' % (i+1), env, front_desk_operator, expert_operator, action_break_generator)  

#### Break Generator Function

In [ ]:
def break_generator(env, front_desk_operator, expert_operator):
    try:
        while True:
            duration = numpy.random.poisson(MEAN_BREAK) #Duration of a break is distributed according to a Poisson distribution
            yield env.timeout(duration)
            print("Expert operator has dediced to give break at", env.now)
            with expert_operator.request(priority = LOW) as req:
                yield req
                print("Expert operator has started to the break at", env.now)
                yield env.timeout(BREAK_DURATION) #Add the break duration to the current time
                print("Expert operator has finished the break at", env.now)
    except simpy.Interrupt:
        print('Expert will no longer give breaks')


#### Running of the environment

In [ ]:
env = simpy.Environment()
front_desk_operator = simpy.Resource(env, capacity = 1) #There is only one front desk operator
expert_operator = simpy.PriorityResource(env, capacity = 1) #There is only one expert operator
action_break_generator = env.process(break_generator(env, front_desk_operator, expert_operator)) #Process the break operation
env.process(customer_generator(env, front_desk_operator, expert_operator, action_break_generator)) #Generate the customers

env.run()